In [1]:
import numpy as np
from math import exp
def Xmodified(X):
    #input1  : designmatrix X
    #output1 : modified designmatrix --> column containing ones is added
    #function modifies the input designmatrix
    # column with ones is added 
    rows = X.shape[0]
    onearray = np.ones([1,rows])
    Xt = np.transpose(X)
    Y=np.concatenate((onearray,Xt))
    Z = np.transpose(Y) 
    return Z

def probability(beta,x): 
    # input1 :  parameter beta
    # input2 :  predictors (in our case spin variables)
    # output1 : probability for state y = 1 given beta and x 
    
    expo = beta.dot(np.transpose(x))
    y = np.exp(expo)/(1+np.exp(expo))
    return y




def matrixW(beta,X):       
    #input1 : parameter beta
    #input2 : Designmatrix X 
    #output1: diagonal matrix p(y=1|beta,X[k,:])*p(y=0|beta,X[k,:]) as kth diagonal element
    
    rows = X.shape[0]
    columns = X.shape[1]
    
    #initializing the diagonal
    diagonale = np.zeros([rows])
    
    #inserting values
    for j in range(0,rows):
        x = X[j,:]
        p = probability(beta,x)    
        diagonale[j]= p*(1-p)
        
    #creating diagonal matrix based on diagonal values
    W=np.diag(diagonale)
    return W

def secondDerivative(beta,X):
    #input1 : parameter beta
    #input2 : designmatrix X
    #output1: second derivative of the cost function (Hessian) 
    
    W = matrixW(beta,X)
    Xtranspose = np.transpose(X)
    D2 = (Xtranspose.dot(W)).dot(X)
    return D2

def firstDerivative(beta,X,y):
    #input1  : parameter beta
    #input2  : designmatrix X
    #input3  : target values y
    #output1 : first derivative of the cost function
    
    #generating factor (y-p) as described in lecture slides 
    length = y.size
    factor = np.ones([length])
    for k in range(0,length):
        x = X[k,:]
        prob1 = probability(beta,x)
        prob0 = 1 - prob1
        factor[k] = y[k] - (y[k]*prob1 + (1-y[k])*prob0)
        
    D = -np.transpose(X).dot(factor)
    return D

def NewtonRaphson(X,y,beta,condition,iteration): 
    #input1 = designmatrix X
    #input2 = targetvalues
    #input3 = initial parameter beta
    #input4 = condition for difference between consecutive betas
    #input5 = max iterations
    #output1 = optimal parameter beta
    
    # initialization of betanew and betaold and calculation of needed properties 
    betanew = beta 
    betaold = betanew
    D2inv   = np.linalg.pinv(secondDerivative(betaold,X))
    D1      = firstDerivative(beta,X,y)
    subtractor = D2inv.dot(D1)
    betanew = betaold - subtractor

    # iterate until ||(betaold-betanew)||< condition or we reach max of desired iterations
    s = 0
    while(s<iteration)and(np.linalg.norm(np.array(betaold)-np.array(betanew))>condition):
        betaold=betanew
        D2inv = np.linalg.pinv(secondDerivative(betaold,X))
        D1 = firstDerivative(beta,X,y)
        subtractor = D2inv.dot(D1)
        betanew = betaold - subtractor
        s = s+1
    return betanew

def Classifier(beta,x):
    #input1 : parameter beta
    #input2 : x values (for example from training set)
    #output : predicted class (based on soft classifier)
    
    p = beta.dot(np.transpose(x))
    
    #because exp(z)/1+exp(z) is monotonously increasing we get
    # z>0 => exp(z)/1+exp(z) > 0.5 
    if (p>0):
        selected_class = 1
        return selected_class
    if(p<=0):
        selected_class = 0
        return selected_class
    
def Accuracy(y,beta,Xtest):
    #input1 : target value
    #input2 : parameter beta
    #input3 : designmatrix of data
    #output1: Accuracy score
    
    #adding column containing ones to the designmatrix
    X = Xmodified(Xtest)
    
    rows = X.shape[0]
    columns= X.shape[1]
    length = rows
    
    #initializing ypredict
    ypredict = np.zeros(length)
    
    #filling ypredict with the predicted classes
    for j in range(0,length):
        ypredict[j] = Classifier(beta,X[j,:])
        
    #generating vector correct with : correct[k] = 1 if kth predticion is correct
    #                                 correct[k] = 0 if kth prediction is incorrect
    correct = 1-(abs(y-ypredict))
    
    #calculating percentage of correct predictions
    percentage = sum(correct)/rows
    return percentage    

In [2]:
import numpy as np

import warnings
#Comment this to turn on warnings
warnings.filterwarnings('ignore')

np.random.seed() # shuffle random seed generator

# Ising model parameters
L=40 # linear system size
J=-1.0 # Ising interaction
T=np.linspace(0.25,4.0,16) # set of temperatures
T_c=2.26 # Onsager critical temperature in the TD limit

##### prepare training and test data sets

import pickle,os
from sklearn.model_selection import train_test_split

###### define ML parameters
num_classes=2
train_to_test_ratio=0.5 # training samples

path_to_data=os.path.expanduser('~')+'/Desktop/Uni/Oslo/MachineLearning/project2/'
file_name = "Ising2DFM_reSample_L40_T=All.pkl"
data = pickle.load(open(path_to_data+file_name,'rb'))
data = np.unpackbits(data).reshape(-1, 1600) # Decompress array and reshape for convenience
data=data.astype('int')
data[np.where(data==0)]=-1 # map 0 state to -1 (Ising variable can take values +/-1)

file_name = "Ising2DFM_reSample_L40_T=All_labels.pkl" # this file contains 16*10000 samples taken in T=np.arange(0.25,4.0001,0.25)
labels = pickle.load(open(path_to_data+file_name,'rb')) # pickle reads the file and returns the Python object (here just a 1D array with the binary labels)

# divide data into ordered, critical and disordered
X_ordered=data[:70000,:]
Y_ordered=labels[:70000]

X_critical=data[70000:100000,:]
Y_critical=labels[70000:100000]

X_disordered=data[100000:,:]
Y_disordered=labels[100000:]

del data,labels

# define training and test data sets
X=np.concatenate((X_ordered,X_disordered))
Y=np.concatenate((Y_ordered,Y_disordered))

# pick random data points from ordered and disordered states 
# to create the training and test sets
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,train_size=train_to_test_ratio)

# full data set
X=np.concatenate((X_critical,X))
Y=np.concatenate((Y_critical,Y))

#print('X_train shape:', X_train.shape)
#print('Y_train shape:', Y_train.shape)
#print()
print(X_train.shape[0], 'train samples')
print(X_critical.shape[0], 'critical samples')
print(X_test.shape[0], 'test samples')

65000 train samples
30000 critical samples
65000 test samples


In [3]:
predictors = 8
trainsize  = 30000
testsize   = 60000
X_try = X_train[0:trainsize,0:predictors]
Y_try = Y_train[0:trainsize]
X = Xmodified(X_try)
beta = np.ones([1,predictors+1])
betapredict = NewtonRaphson(X,Y_try,beta,0.00001,1)
Y_test2 = Y_test[0:testsize]
X_test2 = X_test[0:testsize,0:predictors]
acc = Accuracy(Y_test2,betapredict,X_test2)

print('Number of predictors :'+str(predictors))
print('Trainsize            :'+str(trainsize))
print('Testsize             :'+str(testsize))
print('Accuracy             :'+str(acc))

Number of predictors :8
Trainsize            :30000
Testsize             :60000
Accuracy             :0.5294666666666666
